In [8]:
from pystac_client import Client
from odc.stac import load
from odc.algo import geomedian_with_mads
import geopandas as gpd
from dea_tools.coastal import pixel_tides

In [2]:
study_area = "9,19"
grid_path = "https://raw.githubusercontent.com/auspatious/dea-coastlines/stable/data/raw/vietnam_tiles.geojson"
stac_catalog = "https://earth-search.aws.element84.com/v1/"
stac_collection = "sentinel-2-l2a"
datetime = "2023"

client = Client.open(stac_catalog)

In [3]:
gridcell_gdf = gpd.read_file(grid_path).to_crs(epsg=4326).set_index("id")
gridcell_gdf.index = gridcell_gdf.index.astype(str)
geometry = gridcell_gdf.loc[[study_area]]

In [5]:
items = list(client.search(
    collections=[stac_collection],
    bbox=list(geometry.bounds.values[0]),
    datetime=datetime,
).items())

/var/folders/p8/zk3xfwk969vdx8d5284hrgww0000gn/T/ipykernel_87232/1536206681.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bbox=list(geometry.buffer(0.05).bounds.values[0]),


In [6]:
print(f"Found {len(items)} items")

Found 270 items


In [10]:
data = load(items, measurements=["red", "green", "blue", "scl"], resolution=100, chunks=dict(x=2048, y=2048), groupby="solar_day")
data

<xarray.Dataset>
Dimensions:      (y: 2099, x: 2099, time: 68)
Coordinates:
  * y            (y) float64 1.1e+06 1.1e+06 1.1e+06 ... 8.904e+05 8.902e+05
  * x            (x) float64 4e+05 4e+05 4.002e+05 ... 6.096e+05 6.098e+05
    spatial_ref  int32 32648
  * time         (time) datetime64[ns] 2023-01-05T03:35:28.291000 ... 2023-12...
Data variables:
    red          (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [12]:
tides_lowres = pixel_tides(data, resample=False, directory="~/Data/tide_models_clipped")

# Calculate the lowest and highest 10% of tides
lowest_20, highest_20 = tides_lowres.tide_m.quantile([0.2, 0.8]).values

# Filter our data to low and high tide observations
filtered_low = data.where(tides_lowres.tide_m <= lowest_20, drop=True)
filtered_high = data.where(tides_lowres.tide_m >= highest_20, drop=True)


Creating reduced resolution 5000 x 5000 metre tide modelling array
Modelling tides using FES2014 in parallel


100%|██████████| 5/5 [00:18<00:00,  3.73s/it]


Returning low resolution tide array


AttributeError: 'DataArray' object has no attribute 'tide_m'

In [ ]:
tide_cutoff_min, tide_cutoff_max = tide_cutoffs(
    ds, tides_lowres, tide_centre=tide_centre, reproject=False
)

extreme_tides = (tides_lowres <= tide_cutoff_min) | (
    tides_lowres >= tide_cutoff_max
)

# Filter out empty scenes
filtered = ds.sel(time=~extreme_tides.all(dim=["x", "y"]))

# Check we have some scenes remaining after!
if len(filtered.time) == 0:
    raise CoastlinesException("No data remains after filtering for extreme tides.")

return filtered

In [ ]:
geomad = geomedian_with_mads(
    data, num_threads=self.num_threads, work_chunks=self.work_chunks
)